In [1]:
# Aim: Scrape data from Wikipedia to create a text corpus
# Process:
# Result:

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
class WikiCategoryDataScrapper:
    def __init__(self, url, articlePrefix):
        self.categoryURL = url
        self.articlePrefix = prefix
        self.categoryPageSoup = self.soup(self.request(self.categoryURL))
        self.index = self.letterIndices()
        self.meta = {}
        print('Identifying articles to scrape.')
        for letter in tqdm(self.index.keys()):
            self.meta[letter] = self.letterLinks(letter)
    
    def request(self,url):
        response = requests.get(url)
        return response

    def soup(self, response):
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    
    def letterIndices(self):
        index = {}
        soups = self.categoryPageSoup.body.find(id='mw-pages').find(class_='CategoryIndex').find_all('li')[2:]
        for soup in soups:
            index[soup.text] = soup.a['href']
        return index
    
    def letterLinks(self,letter):
        letterResponse = self.request(self.index[letter])
        letterSoup = self.soup(letterResponse).body.find(class_='mw-category-group')
        articles =  {}
        articleSoups = letterSoup.find_all('li')
        for articleSoup in articleSoups:
            articles[articleSoup.text] = self.articlePrefix + articleSoup.a['href']
        return articles

In [ ]:
url = 'https://fr.wikipedia.org/w/index.php?title=Cat%C3%A9gorie:Portail:Robotique/Articles_li%C3%A9s&pageuntil=Essaim+de+drones#mw-pages'
prefix = 'https://fr.wikipedia.org'
scrapper = WikiCategoryDataScrapper(url,prefix)

Identifying articles to scrape.


 85%|█████████████████████████████████████████████████████████████████████▍            | 22/26 [00:13<00:02,  1.73it/s]

In [ ]:
scrapper.meta